In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/a-bit-better/__results__.html
/kaggle/input/a-bit-better/bbox3
/kaggle/input/a-bit-better/submission.csv
/kaggle/input/a-bit-better/__notebook__.ipynb
/kaggle/input/a-bit-better/__output__.json
/kaggle/input/a-bit-better/custom.css
/kaggle/input/new-year-same-old-bbox/__results__.html
/kaggle/input/new-year-same-old-bbox/bbox3
/kaggle/input/new-year-same-old-bbox/submission.csv
/kaggle/input/new-year-same-old-bbox/__notebook__.ipynb
/kaggle/input/new-year-same-old-bbox/__output__.json
/kaggle/input/new-year-same-old-bbox/custom.css
/kaggle/input/mysteriously-slightly-better-solution/__results__.html
/kaggle/input/mysteriously-slightly-better-solution/submission.csv
/kaggle/input/mysteriously-slightly-better-solution/__notebook__.ipynb
/kaggle/input/mysteriously-slightly-better-solution/__output__.json
/kaggle/input/mysteriously-slightly-better-solution/custom.css
/kaggle/input/santa-2025/sample_submission.csv
/kaggle/input/santa-submission-happy-new-year/__results__.html


In [2]:
import subprocess
import re
import shutil
import os

# --- 1. Setup Paths & Copy Files ---
INPUT_SUB = '/kaggle/input/magic-fast-backprop/submission.csv'
INPUT_BIN = '/kaggle/input/a-bit-better/bbox3'
WORKING_DIR = '/kaggle/working/'

print("📂 Setting up environment...")

# Copy submission to working directory
if os.path.exists(INPUT_SUB):
    shutil.copy(INPUT_SUB, os.path.join(WORKING_DIR, 'submission.csv'))
    print(f"✅ Copied {INPUT_SUB} -> ./submission.csv")

# Copy executable to working directory
if os.path.exists(INPUT_BIN):
    shutil.copy(INPUT_BIN, os.path.join(WORKING_DIR, 'bbox3'))
    print(f"✅ Copied {INPUT_BIN} -> ./bbox3")

# Set executable permissions
os.chmod('./bbox3', 0o755)
print("✅ Permissions set for ./bbox3")

import math
import pandas as pd
import numpy as np
from numba import njit

@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    return x,y

@njit
def score_group(xs,ys,degs,tx,ty):
    n=xs.size; V=tx.size
    mnx=1e300; mny=1e300; mxx=-1e300; mxy=-1e300
    for i in range(n):
        r=degs[i]*math.pi/180.0
        c=math.cos(r); s=math.sin(r)
        xi=xs[i]; yi=ys[i]
        for j in range(V):
            X=c*tx[j]-s*ty[j]+xi
            Y=s*tx[j]+c*ty[j]+yi
            if X<mnx: mnx=X
            if X>mxx: mxx=X
            if Y<mny: mny=Y
            if Y>mxy: mxy=Y
    side=max(mxx-mnx,mxy-mny)
    return side*side/ n

def strip(a):
    return np.array([float(str(v).replace("s","")) for v in a],np.float64)

def all_csv_files():
    out=[]
    for f in subs:
        out += glob.glob(f + "/*.csv", recursive=False)
    return out

from decimal import Decimal, getcontext

from shapely import affinity
from shapely.geometry import Polygon

# Set precision for Decimal
getcontext().prec = 25
scale_factor = Decimal("1e18")

class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x="0", center_y="0", angle="0"):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal("0.15")
        trunk_h = Decimal("0.2")
        base_w = Decimal("0.7")
        mid_w = Decimal("0.4")
        top_w = Decimal("0.25")
        tip_y = Decimal("0.8")
        tier_1_y = Decimal("0.5")
        tier_2_y = Decimal("0.25")
        base_y = Decimal("0.0")
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                # Start at Tip
                (Decimal("0.0") * scale_factor, tip_y * scale_factor),
                # Right side - Top Tier
                (top_w / Decimal("2") * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal("4") * scale_factor, tier_1_y * scale_factor),
                # Right side - Middle Tier
                (mid_w / Decimal("2") * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal("4") * scale_factor, tier_2_y * scale_factor),
                # Right side - Bottom Tier
                (base_w / Decimal("2") * scale_factor, base_y * scale_factor),
                # Right Trunk
                (trunk_w / Decimal("2") * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal("2") * scale_factor, trunk_bottom_y * scale_factor),
                # Left Trunk
                (-(trunk_w / Decimal("2")) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal("2")) * scale_factor, base_y * scale_factor),
                # Left side - Bottom Tier
                (-(base_w / Decimal("2")) * scale_factor, base_y * scale_factor),
                # Left side - Middle Tier
                (-(mid_w / Decimal("4")) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal("2")) * scale_factor, tier_2_y * scale_factor),
                # Left side - Top Tier
                (-(top_w / Decimal("4")) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal("2")) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated, xoff=float(self.center_x * scale_factor), yoff=float(self.center_y * scale_factor)
        )

import pandas as pd
import numpy as np
from decimal import Decimal
from shapely.strtree import STRtree


def to_str(x: Decimal):
    return f"s{float(x)}"


def load_configuration_from_df(n, existing_df):
    """
    Load existing configuration from submission CSV.
    """
    group_data = existing_df[existing_df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = row["x"][1:]  # Remove 's' prefix
        y = row["y"][1:]
        deg = row["deg"][1:]
        trees.append(ChristmasTree(x, y, deg))
    return trees

def get_score(trees, n=None):
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e18 for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    score = max(max_x - min_x, max_y - min_y) ** 2
    if n is not None:
        score /= n
    return score

def has_overlap(trees):
    """Check if any trees overlap"""
    if len(trees) <= 1:
        return False

    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)

    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

import pandas as pd

def eval_df_sub(df, verb):
    failed = []
    total_score = 0.0
    for n in range(1, 201):
        trees = load_configuration_from_df(n, df)
        score = get_score(trees, n)
        total_score += score
        if verb:
            print(f"{n:3}  {score:.6f}")
        if has_overlap(trees):
            failed.append(n)

    if len(failed) == 0:
        print("Overlap check was succesfull")
    else:
        print("Overlap check failed for", *failed)

    print(f"Total score: {total_score:.12f}")
    return total_score


IMPROVEMENT_TARGET = 0.00001

def main():
    # Read in current ensemble output and check for overlaps
    df = pd.read_csv("submission.csv")
    target_score = eval_df_sub(df, False)
    new_score = target_score
    target_score = target_score - IMPROVEMENT_TARGET
    
    print("Starting bbox3 runs")
    
    import subprocess
    import random
    
    while new_score >= target_score:
        random.seed() # Use a random seed every time; add fixed number for repeatability
        n =  str(random.randint(100,1000))
        r = str(random.randint(10,100))
        print(f"#### Running bbox3 for n={n} r={r}")
        subprocess.run(["./bbox3", 
                        "-n", str(random.randint(100,1000)), 
                        "-r", str(random.randint(10,100))])
        df = pd.read_csv("submission.csv")
        new_score = eval_df_sub(df, False)

main()

📂 Setting up environment...
✅ Copied /kaggle/input/magic-fast-backprop/submission.csv -> ./submission.csv
✅ Copied /kaggle/input/a-bit-better/bbox3 -> ./bbox3
✅ Permissions set for ./bbox3
Overlap check was succesfull
Total score: 70.897092480731
Starting bbox3 runs
#### Running bbox3 for n=186 r=14
Loading submission.csv...
Loaded 200 configs (N=1 to N=200).
Initial Total Score: 70.897092

Starting parallel optimization on 200 configurations.
Using 4 threads (Max Concurrency).

[0] n=194: 0.340203053616 -> 0.340203033891 (0.0000% better) 🏆
[0] n=193: 0.340165565244 -> 0.340165475389 (0.0000% better) 🏆
[3] n=110: 0.337619638694 -> 0.337619581162 (0.0000% better) 🏆
[0] n=181: 0.330200318761 -> 0.330199412433 (0.0003% better) 🏆
[0] n=178: 0.335408729016 -> 0.335408533716 (0.0001% better) 🏆
[0] n=177: 0.337302986613 -> 0.337302684035 (0.0001% better) 🏆
[3] n=107: 0.346235106577 -> 0.346234964177 (0.0000% better) 🏆
[0] n=155: 0.332096894537 -> 0.332096860713 (0.0000% better) 🏆
[0] n=152: 0